In [8]:
import numpy as np
import matplotlib.pyplot as plt

import chainer.optimizers as Opt
import chainer.functions as F
import chainer.links as L
import chainer.datasets as ds
import chainer.dataset.convert as con
from chainer.iterators import SerialIterator as siter
from chainer import Variable,Chain,config,cuda

import princess as ohm

In [9]:
train,test = ds.get_cifar10()
xtrain,ttrain = con.concat_examples(train)
xtest,ttest = con.concat_examples(test)

In [10]:
Dtrain,ch,Ny,Nx = xtrain.shape
print(Dtrain,ch,Ny,Nx)

50000 3 32 32


In [12]:
class ResBlock(Chain):
    def __init__(self, ch, bn=True):
        layers = {}
        layers['conv1'] = L.Convolution2D(ch,ch,3,1,1)
        layers['conv2'] = L.Convolution2D(ch,ch,3,1,1)
        layers['bnorm1'] = L.BatchNormalization(ch)
        layers['bnorm2'] = L.BatchNormalization(ch)
        super().__init__(**layers)
        
    def __call__(self,x):
        h = self.conv1(x)
        if self.bn == True:
            h = self.bnorm1(h)
        h = F.relu(h)
        h = self.conv2(h)
        if self.bn == True:
            h = self.bnorm2(h)
        h = h +x
        h = F.relu(h)

        return h

In [13]:
class Bottleneck(Chain):
    def __init__(self, ch, bn=True):
        layers = {}
        layers['conv1'] = L.Convolution2D(ch,ch,1,1,0)
        layers['conv2'] = L.Convolution2D(ch,ch,3,1,1)
        layers['conv3'] = L.Convolution2D(ch,ch,1,1,0)
        layers['bnorm1'] = L.BatchNormalization(ch)
        layers['bnorm2'] = L.BatchNormalization(ch)
        layers['bnorm3'] = L.BatchNormalization(ch)
        super().__init__(**layers)
        
    def __call__(self,x):
        h = self.conv1(x)
        if self.bn == True:
            h = self.bnorm1(h)
        h = F.relu(h)
        h = self.conv2(h)
        if self.bn == True:
            h = self.bnorm2(h)
        h = F.relu(h)
        h = self.conv3(h)
        if self.bn == True:
            h = self.bnorm3(h)
        h = h +x
        h = F.relu(h)

        return h

In [14]:
class PixelShuffler(Chain):
    def __init__(self,ch,r):
        self.r = r
        self.ch = ch
        super().__init__()
    
    def __call__(self,x):    
        batchsize,ch,Ny,Nx = x.shape
        ch_y = ch//(self.r**2)
        Ny_y = Ny*self.r
        Nx_y = Nx*self.r
        h = F.reshape(x, (batchsize, self.r, self.r, ch_y, Ny, Nx))
        h =  F.transpose(h, (0, 3, 4, 1, 5, 2))
        y = F.reshape(h, (batchsize, ch_y, Ny_y, Nx_y))
        return y

In [6]:
class CBR(Chain):
    def __init__(self, ch_in, ch_out, sample='down',bn=True,  act=F.relu, drop=False):
        self.bn = bn
        self.act = act
        self.drop = drop

        layers = {}
        if sample=='down':
            layers['conv'] = L.Convolution2D(ch_in, ch_out, 4, 2, 1)
        else:
            layers['conv'] = L.Deconvolution2D(ch_in, ch_out, 4, 2, 1)
        if bn:
            layers['bnorm'] = L.BatchNormalization(ch_out)
        super().__init__(**layers)
        
    def __call__(self, x):
        h = self.conv(x)
        if self.bn == 1:
            h = self.bnorm(h)
        if self.drop == 1:
            h = F.dropout(h)
        h = self.act(h)
        return h

In [7]:
class CNN(Chain):
    def __init__(self, ch_in,ch_out):
        initializer = Ini.HeNormal()
        layers = {}
        layers['conv1'] = L.Convolution2D(ch_in,ch_out,ksize=3,stride=2,pad=1)
        layers['bnorm1'] = L.BatchNormalization(ch_out)
        super().__init__(**layers)
        
    def __call__(self,x):
        h = self.conv1(x)
        h = self.bnorm1(h)
        h = F.relu(h)
        #h = F.max_pooling_2d(h,ksize=3,stride=2)

        return h

In [8]:
C = ttrain.max() + 1
H1 = 64
H2 = 64
H3 = 64

NN = Chain(cnn1 = CNN(ch,H1),
           cnn2 = CNN(H1,H2),
           Rb1 = ResBlock(H2),
           Rb2 = ResBlock(H2),
                 l1=L.Linear(None,H3,initialW=initializer),
                 l2=L.Linear(H3,C,initialW=initializer),
                )


In [16]:
Ny = 31
Nx = 31

In [17]:
x = np.random.rand(ch*Ny*Nx).reshape(1,ch,Ny,Nx).astype(np.float32)
ohm.check_network(x,L.Convolution2D(ch,10,ksize=4,stride=2,pad=1))


input: (1, 3, 31, 31)
output: (1, 10, 15, 15)


In [10]:
def model(x):
    h = NN.cnn1(x)
    h = NN.cnn2(h)
    h = NN.Rb1(h)
    h = NN.Rb2(h)
    
    h = F.dropout(NN.l1(h),0.5)
    h = F.relu(h)
    y = NN.l2(h)
    return y

In [ ]:
gpu_device = 2
cuda.get_device(gpu_device).use()
NN.to_gpu(gpu_device)

In [ ]:
optNN = Opt.Adam()
optNN.setup(NN)

In [ ]:
train_loss = []
train_acc = []
test_loss = []
test_acc = []
result = [train_loss,train_acc,test_loss,test_acc]

In [ ]:
import pylab as pl
from IPython import display
nepoch = 50
%matplotlib inline
import PIL.Image as im


In [ ]:
def transform_shift_labeled(labeled_data,h_shift=True,v_shift=True):
    data, label = labeled_data
    
    [ch,Ny,Nx] = data.shape
    z_h = Ny*(2.0*np.random.rand(1)-1.0)*h_shift*0.2
    z_v = Nx*(2.0*np.random.rand(1)-1.0)*v_shift*0.2
    data = np.roll(data,int(z_h),axis=1)
    data = np.roll(data,int(z_v),axis=2)
    
    return data, label

In [ ]:
def transform_flip_labeled(labeled_data,h_flip=True,v_flip=True,rotate=True):
    data, label = labeled_data
    #horizontal flip
    z = np.random.randint(2)
    if z*h_flip == 1:
        data = data[:,::-1,:]
    
    #vertical flip
    z = np.random.randint(2)
    if z*v_flip == 1:
        data = data[:,:,::-1]

    #rotate
    z = np.random.randint(2)
    if z*rotate== 1:
        data = data.transpose(0,2,1)
        
    return data, label

In [ ]:
batch = ds.TransformDataset(train, transform_flip_labeled)
plt.imshow(batch[0][0].transpose(1,2,0))
plt.show()

In [ ]:
from tqdm import tqdm

In [ ]:
batch_size = 10000
train_iter = siter(train, batch_size)    
with tqdm(total=nepoch) as pbar:
    while train_iter.epoch < nepoch:
        batch = train_iter.next()
        batch = ds.TransformDataset(batch, transform_flip_labeled)
        batch = ds.TransformDataset(batch, transform_shift_labeled)
        xtrain,ttrain = con.concat_examples(batch)
        data = cuda.to_gpu([xtrain,xtest,ttrain,ttest])
        ohm.learning_process_classification(model,optNN,data,result,100)
        if train_iter.is_new_epoch == 1:
            display.clear_output(wait=True)
            ohm.plot_result2(result[0],result[1],'loss function in training','step','loss function',0.0,4.0)
            ohm.plot_result2(result[2],result[3],'Accuracy in training and test','step','accuracy')
            display.display(pl.gcf())
        pbar.update(train_iter.is_new_epoch)

In [ ]:
ohm.plot_result2(result[0],result[1],'loss function in training','step','loss function',0.0,4.0)
ohm.plot_result2(result[2],result[3],'Accuracy in training and test','step','accuracy')

In [ ]:
max(result[3][:])